<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

Cloning into 'Project9_data'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 121 (delta 42), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (121/121), 12.87 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [45]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re

!ls Project9_data


'autisterna om kvinnor på spektrat.conllu'  'jag upphäver gravitationen.conllu'
'Autisterna Om kvinnor på spektrat.txt'     'Jag upphäver gravitationen.txt'
'en annorlunda barndom.conllu'		    'konsten att fejka arabiska.conllu'
'En annorlunda barndom.txt'		    'Konsten att fejka arabiska.txt'
'ett annorlunda liv.conllu'		     Project.ipynb
'Ett annorlunda liv.txt'		    'vanliga människor autistens guide till galaxen.conllu'
 expressen.conllu			    'Vanliga människor autistens guide till galaxen.txt'
 expressen.txt


In [46]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...
INFO:stanza:File exists: /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [47]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu".lower()
  CoNLL.write_doc2conll(doc, conllufilename)

  return conllufilename

In [74]:
class SemanticAnalyzer:
  def __init__(self, file_path):
    self.file_path = file_path
    self.data = self.parse_conll_u(self.file_path)
    self.PFL_sentence_matches = self.person_first_lang(self.data)
    self.IFL_sentence_matches = self.identity_first_lang(self.data)

  def parse_conll_u(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
      data = {}
      sentence = []
      for line in file:
        line = line.strip()
        if line.startswith('# text'):
          sentence = []
        if line.startswith("# sent_id"):
          line = line.split("=")
          key = line[1].strip()
          sentence = []
        elif line == '':
          if sentence:
            data[key] = sentence
        else:
            sentence.append(line.split('\t'))
    return data

  def person_first_lang(self, data):
    PFL_sents = {}
    PFL_regx = re.compile(r"ha|med", re.IGNORECASE)

    for id, sentence in data.items():
      for token in sentence:
        if token[1] == "autism" or token[1].lower() == "autismspektrumtillstånd" or token[1].lower() == "ast" or token[1].lower() == "autismdiagnos":
          if sentence[int(token[0])-2][1] == "och".lower():
            if PFL_regx.match(sentence[int(sentence[int(token[6])-1][int(6)])-1][1]): #Huvudet till huvudet till ordet
              PFL_sents[id] = sentence
          if PFL_regx.match(sentence[int(token[6])][2].lower()): #Ordet innan lemma
            PFL_sents[id] = sentence
    return PFL_sents

  def identity_first_lang(self, data):
    IFL_sents = {}
    IFL_regx = re.compile(r"^.*(autistisk|autistiska)", re.IGNORECASE)
    for id, sentence in data.items():
      for token in sentence:

        #Om ordet är autistiska/autistisk låt användaren manuellt kolla om det är ifl
        if IFL_regx.match(token[1].lower()) and token[1] != "autistiskt":
          if sentence[int(token[0])][7] in ("nsubj:pass", "nsubj"):
            print(sentence[int(token[0])]) #continue HERREEEEEEE
          test = ' '.join(sublist[1] for sublist in sentence)
          print(test)
          while True:
            question = input("Is this ifl (y/n): ")
            if question.lower() not in ('y', 'n'):
                print("Not an appropriate choice.")
                print(test)
            else:
              if question.lower() == "y":
                IFL_sents[id] = sentence
                break
              if question.lower() == "n":
                break

        #Alla ord som är ifl oavsett kontext
        if token[1] == "autist" or token[1] == "autisten" or token[1] == "autister":
          IFL_sents[id] = sentence

        #på spektrat är ifl
        if token[1] == "spektrumet" or token[1] == "autismspektrumet" or token[1] == "spektrat":
          if sentence[int(token[6])][1].lower() == "på": #Ordets huvud
            IFL_sents[id] = sentence
            continue

    return IFL_sents

####

def main(file, lang):
  analyzer = SemanticAnalyzer(file_path= file)
  if lang.lower() == "pfl":
    matches = analyzer.PFL_sentence_matches
  elif lang.lower() == "ifl":
    matches = analyzer.IFL_sentence_matches
  else:
    print("Invalid language choice. Please choose either 'PFL' or 'IFL'.")
    return

  for id, sentence in matches.items():
    print(f"Sentence ID: {int(id) + 1} | {' '.join(token[1] for token in sentence)}")
    print("")

  print(f"Total {lang} matching sentences: {len(matches)}")

if __name__ == "__main__":
    #file_path = input("Enter the file path: ").lower()
    #if ".conllu" not in file_path:
      #tag_and_convert(file_path)
      #file_path = tag_and_convert(file_path)
    #lang = input("Enter the language (PFL or IFL): ")
    file_path = r'/content/Project9_data/ett annorlunda liv.conllu'
    main(file_path, "ifl")
    main(file_path, "pfl")

['35', 'barn', 'barn', 'NOUN', 'NN|NEU|PLU|IND|NOM', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Plur', '36', 'nsubj:pass', '_', 'start_char=3465|end_char=3469|ner=O']
Diagnosen autism användes för första gången i historien bara några år innan jag föddes , och när jag var barn var det ingen hos oss som hade hört talas om den , och autistiska barn bedömdes felaktigt som utvecklingsstörda .


KeyboardInterrupt: Interrupted by user